<h1>Getting engagement data for tweets Twitter Climate Change sentiment dataset</h1>

In [1]:
import pandas as pd
import numpy as np
import tweepy 
import os

<h2>Load Data</h2>

In [2]:
tweets = pd.read_csv("../twitter_sentiment_data.csv")

In [3]:
tweets.head()

,sentiment,message,tweetid
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153


<h2>Create Tweepy Client</h2>

In [34]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAHc8OQEAAAAAcU%2FIXW1wjJ0h%2FpQxvRKbYDyhJVI%3DntLxP3rMhKb9yTzTqLekvbsl2bP4F1OCRN4DLw3JPSbn9XzbDq"
client = tweepy.Client(bearer_token, wait_on_rate_limit = True)

<h2>Extract Engagement</h2>

In [35]:
def extract_entities(t):
    keys = list(t.data.entities.keys())
    if 'mentions' in keys:
        mentioned_users = [i['username'] for i in t.data.entities['mentions']]
    else:
        mentioned_users = None
    if 'hashtags' in keys:
        hashtags = [i['tag'] for i in t.data.entities['hashtags']]
    else: 
        hashtags = None
    if 'annnotations' in keys:
        annotations = [i['annotation'] for i in t.data.entities['annotations']]
    else: 
        annotations = None
    if 'urls' in keys:
        urls = [i['url'] for i in t.data.entities['urls']]
    else: 
        urls = None
    return (mentioned_users, hashtags, annotations, None)
        

In [36]:
def get_tweet_data(t):
    t = client.get_tweet(t, tweet_fields=['entities', 'public_metrics', 'author_id'])
    if t.data == None:
        return []
    #identifiers
    tweet_id = t.data.id
    tweeter_id = t.data.author_id
    #get tweet attributes
    if getattr(t.data, 'public_metrics') != None:
        metrics = t.data.public_metrics
        retweets = metrics['retweet_count']
        replies = metrics['reply_count']
        likes = metrics['like_count']
        quote_tweets = metrics['quote_count']
    else:
        retweets = None
        replies = None
        likes = None
        quote_tweets = None
    #get tweet entities
    if getattr(t.data, 'entities'):
        mentioned_users, hashtags, annotations, urls = extract_entities(t)
    else:
        mentioned_users, hashtags, annotations, urls = None, None, None, None 
    #write to file
    data = np.array([tweet_id, tweeter_id, retweets, replies, likes, quote_tweets, mentioned_users,
                     hashtags, annotations, urls], dtype=object)
    data_df = pd.DataFrame([data], columns = ['tweet_id', 'tweeter_id', 'retweets', 'replies', 'likes', 
                                              'quote_tweets', 'mentioned_users', 'hashtags', 'annotations', 
                                              'urls'])
    try:
        data_df.to_csv("tweet_engagement.csv", mode="a", index=False, header=True)
    except:
        return None
    return None

In [37]:
ids = tweets['tweetid'].values[100:1000]
engagement_data = [get_tweet_data(i) for i in ids]

Rate limit exceeded. Sleeping for 852 seconds.
Rate limit exceeded. Sleeping for 846 seconds.


In [28]:
test = pd.read_csv("tweet_engagement.csv")